In [1]:
%pip install nltk
import ollama
import asyncio
import time 
import gc
import sys
sys.path.insert(0, '/home/bfhgfe/agent')
import Tools.search
from Tools.search import search_for_evidence, search_recent
from Tools.ref import QuickMemory
from Tools.clean import clean_structured_output, clean_for_prompt
apathto = open('./apathto.txt', 'r')
current_text = clean_for_prompt(apathto.read())
memory = QuickMemory()
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import nltk
nltk.download('punkt_tab')
# task 1: list claims 
gc.disable()
start_time = time.time()
# Step 1: Extract all claims
sentences = nltk.sent_tokenize(current_text)
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5)
clusters = clustering.fit_predict(embeddings)

# Group sentences by cluster
clustered_chunks = {}
for i, cluster_id in enumerate(clusters):
    if cluster_id not in clustered_chunks:
        clustered_chunks[cluster_id] = []
    clustered_chunks[cluster_id].append(sentences[i])

print("clusters:\n")
print("-----------------------------------------------------")
for x in clustered_chunks:
    print (x)
    for y in clustered_chunks[x]:
        print (clustered_chunks[x]
print("-----------------------------------------------------")

task2 = str.format(f"""
Take this list and copy all the items to a single list - output as JSON ONLY.
List: {clustered_chunks}
""", clustered_chunks)

result = ollama.chat(
    model='qwen2.5:0.5b',
    messages=[{'role': 'user', 'content': task2}],
    options={'format':'json', 'temperature': 0.1, })['message']['content']

cleaned = clean_structured_output(result)
if isinstance(cleaned, list):
    print("\ndistinct claims: \n")
    print(cleaned)

# step 2: Identify implicit claims
task3 = str.format(f"""
Task: Output as a single JSON list only. In your own words, what would need to be true for the claims in this list to be true?
Explicit claims: {cleaned}
""", cleaned)

result = ollama.chat(
    model='qwen2.5:0.5b',
    messages=[{'role': 'user', 'content': task2}],
    options={'format':'json', 'temperature': 0.1, "DRY": 0.3},
    stream=False,
)['message']['content']

implicit = clean_structured_output(result)
if isinstance(cleaned, list):
    print("\nImplicit claims: \n")
    print(implicit)

print("--- %s seconds ---" % (time.time() - start_time))

gc.enable()
gc.collect()




# task 2: verify claims


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


2025-08-23 16:10:27.817885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755990627.834278    2467 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755990627.839141    2467 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755990627.856721    2467 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755990627.856750    2467 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755990627.856752    2467 computation_placer.cc:177] computation placer alr

clusters:

-----------------------------------------------------
1
['A Path to Beneficial Superintelligence by SingularityNet Ambassadors source: https://medium.com/@singularitynetambassadors/a-path-to-beneficial-superintelligence-13ea8dbab2a9  Dr. Ben Goertzel, the founder and CEO of SingularityNET, was welcomed on stage as a leader in artificial general intelligence, robotics, and computational finance research.', 'I’m going to talk about what we’re doing with the artificial superintelligence alliance, which aims to build decentralized AI and decentralized AGI.', 'But what I’m going to talk about now is how we went from AI to artificial general intelligence to artificial superintelligence without screwing up our lives and doing so in a way that benefits humans and other sentient bei...'] : A Path to Beneficial Superintelligence by SingularityNet Ambassadors source: https://medium.com/@singularitynetambassadors/a-path-to-beneficial-superintelligence-13ea8dbab2a9  Dr. Ben Goertzel, the

KeyError: 'np'